In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.data_loader import KeypointsDataset, NormalizeCentered2D, ToTensor, BODY_PARTS
from torch.utils.data import Dataset, DataLoader
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point

from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3
import pandas as pd
import numpy as np
import plotly.express as px
import time
from matplotlib import pyplot as plt
from PIL import Image

from collections import defaultdict
import datetime as dt
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from collections import defaultdict
from matplotlib.ticker import PercentFormatter




<h1> Load Optical Data </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

df = pd.concat([
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/data_dump_1.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-06-from-2019-10-25-to-2019-11-01.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-07-from-2019-11-01-to-2019-11-08.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-08-from-2019-11-08-to-2019-11-15.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-09-from-2019-11-15-to-2019-11-22.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-10-from-2019-11-22-to-2019-11-29.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-11-from-2019-11-29-to-2019-12-06.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-12-from-2019-12-06-to-2019-12-13.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-13-from-2019-12-13-to-2019-12-20.csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/biomass.csv-61-14-from-2019-12-20-to-2019-12-27.csv')
])    



In [ ]:
df = pd.read_csv('/root/data/alok/biomass_estimation/playground/imr_data_kf.csv')

In [ ]:
cols = ['captured_at', 'captured_at_bergen_tz', 'left_crop_url', 'right_crop_url', 'estimated_weight_g', 'estimated_k_factor']
df[cols].to_csv('/root/data/alok/biomass_estimation/playground/imr_modified_data.csv')

In [ ]:
# df.index = pd.to_datetime(df.captured_at)
df.index.tz_convert('Europe/Oslo')

In [ ]:
df['captured_at_bergen_tz'] = df.index.tz_convert('Europe/Oslo')

In [ ]:
df.index = pd.to_datetime(df.captured_at)

In [ ]:
f = '/root/data/alok/biomass_estimation/playground/imr_modified_data.csv'
s3_access_utils.s3_client.upload_file(f, 'aquabyte-images-adhoc-public', 'imr_modified_data.csv')

In [ ]:
df.index = pd.to_datetime(df.captured_at)
df['epoch_ms'] = (df.index.astype(int) / (10**6)).values.astype(int)
df = df.sort_values('epoch_ms', ascending=True).copy(deep=True)

<h1> Load Antenna Registration Data </h1>

In [ ]:
rdf = pd.read_csv('/root/data/alok/biomass_estimation/playground/measure_station_oct_10_2019.csv')
rdf.index = pd.to_datetime(rdf.Read)
rdf.index = rdf.index.tz_localize('Europe/Oslo').tz_convert('UTC')
rdf['epoch_ms'] = (rdf.index.astype(int) / (10**6)).values.astype(int)
rdf = rdf.sort_values('epoch_ms', ascending=True).copy(deep=True)
rdf = rdf[rdf.Ant == '1 a4'].copy(deep=True)

In [ ]:
# get optical lone_epochs
date = '2019-10-04'
epochs = df[date].epoch_ms.values
buffer = 3e3
optical_lone_epochs = []
for idx in range(1, len(epochs)-1):
    if (epochs[idx] - epochs[idx-1] > buffer) and(epochs[idx+1] - epochs[idx] > buffer):
        optical_lone_epochs.append(epochs[idx])
        
optical_lone_epochs = np.array(optical_lone_epochs)
        
# get antenna registration lone epochs
epochs = rdf[date].epoch_ms.values
buffer = 3e3
antenna_lone_epochs = []
for idx in range(1, len(epochs)-1):
    if (epochs[idx] - epochs[idx-1] > buffer) and(epochs[idx+1] - epochs[idx] > buffer):
        antenna_lone_epochs.append(epochs[idx])

antenna_lone_epochs = np.array(antenna_lone_epochs)


        

In [ ]:
matched_epochs = []
for a_epoch in antenna_lone_epochs:
    mask = (optical_lone_epochs > a_epoch - 2e3) & ((optical_lone_epochs < a_epoch + 2e3))
    if optical_lone_epochs[mask].shape[0] == 1:
        o_epoch = optical_lone_epochs[mask][0]
        matched_epochs.append((a_epoch, o_epoch))
print(len(matched_epochs))

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
idx = 1
a_epoch, o_epoch = matched_epochs[idx]
print('Registration timestamp occurred {}s {} image timestamp'.format(1e-3 * abs(a_epoch - o_epoch), 'before' if a_epoch < o_epoch else 'after'))
left_crop_url = df[df.epoch_ms == o_epoch].left_crop_url.iloc[0]
left_crop_f, _, key = s3_access_utils.download_from_url(left_crop_url)
left_crop = Image.open(left_crop_f)
left_raw_im_key = os.path.join(os.path.dirname(key), 'left_frame.resize_512_512.jpg')
left_raw_im_f = s3_access_utils.download_from_s3('aquabyte-frames-resized-inbound', left_raw_im_key)
left_raw_im = Image.open(left_raw_im_f)
left_raw_im

In [ ]:
left_crop

In [ ]:
date = '2019-10-09'
fig, axes = plt.subplots(2, 1, figsize=(10, 10))
axes[0].plot(df[date].epoch_ms.resample('H').agg(lambda x: x.shape[0]), color='blue')
axes[1].plot(rdf[date].epoch_ms.resample('H').agg(lambda x: x.shape[0]), color='red')
axes[0].grid()
axes[1].grid()
plt.show()